(activity9)=
# Activity 9: Positivity in Yeager et al. 2019

**2025-10-03**

---

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets


In [ ]:
# load and examine the data
learning_df = pd.read_csv("~/COMSC-341CD/data/learning_mindset.csv")
learning_df.head()

This selected portion of the National Study of Learning Mindsets dataset is not truly randomized, so we'll need to adjust for confounding.

The columns we will look at are:

- `intervention`: whether the student received the intervention (1) or not (0)
- `success_expect`: student prior mindset about their ability to succeed in school (higher values indicate a stronger belief in their ability to succeed)
- `frst_in_family`: whether the student would be the first in their family to attend college (1) or not (0)
- `gender`: student's self-reported gender
- `school_urbanicity`: categorical variable corresponding to the urbanicity of the school the student attends, e.g. urban, suburban, rural
- `achievement_score`: the student's future grade achievement, standardized such that 0 is the mean and it has a standard deviation of 1

# Part 1: Imbalance in covariates

In this vesrion of the dataset we are anlayzing, there appear to be some differences in the distribution of covariates between the treatment and control groups where participants have different probabilities of receiving the intervention.

## 1.1

Perform two separate `groupby` operations to compute the mean of `intervention`, grouping by: 

- `success_expect`
- `frst_in_family`

What do you observe? Are certain groups of students more or less likely to receive the growth mindset video? Keep in mind that `intervention=1` corresponds to the growth mindset video intervention.

**Your response**: TODO

In [ ]:
# TODO perform two separate groupby operations


---

# Part 2: Examining positivity

In [23]:
covariates = ['success_expect', 'frst_in_family']

## 2.1

Now that we've seeen some potential confounding in `success_expect` and `frst_in_family`, let's try to control for them. If we take the same strategy as we have done before with stratification, we'll need to bin on the confounders and compute treatment effects for each bin.

However, we also need to be careful about positivity violations. First, let's compute the total number of bins we need to create if we want to control for these two covariates.

We can do this by using [pd.Series.nunique](https://pandas.pydata.org/docs/reference/api/pandas.Series.nunique.html) to get the number of unique values for each covariate and then multiplying them together. This is like counting the number of possible bins over the all possible combinations of each variable.

In [ ]:
# TODO calculate the total number of bins 
total_bins = 0

print(f"Total number of bins: {total_bins}")   

## 2.2

Next, let's check if positivity holds. We can do this by grouping over the covariates plus the intervention, and then counting the number of unique groups are actually present in the data.

To generate the per-bin counts, we perform a `groupby(all_cols, as_index=False)` over the intervention and all combinations of the other columns, and the check the `ngroups` attribute of the resulting groupby object. How many groups are there?

In [ ]:
# Group by the intervention column and the two covariates
all_cols = []
#group_count = learning_df.groupby(all_cols, as_index=False).ngroups


print(f"Number actual groups among the bins for {all_cols}: {group_count}")

Since we need each bin to have both control and treatment units in order to have a valid comparison, the total number of groups should be equal to **2 times the total number of bins possible** for there to be no positivity violations.

Does the number of groups you found in 2.1 match this?

**Your response**: TODO

## 2.3

Ideally we'd like to control for as many confounders as possible to make conditional exchangeability more plausible. Let's now add `gender` and `school_urbanicity` to our list of covariates, making a total of 4 confounders.

Repeat the analysis above with the new set of covariates. Do we see positivity violations with the new set of covariates?

**Your response**: TODO


In [ ]:
# TODO your code here 


print(f"Number actual groups among the bins for {all_cols}: {group_count}")



# Optional extra


if we actually want to see the bins that are missing, we can generate a [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) of the counts, and then identify the bins that are missing in either the control or treatment group.

In [ ]:
count_df = learning_df.groupby(all_cols, as_index=False).size()

# Create a pivot table to show counts by intervention and bins
bin_pivot = pd.pivot_table(
    count_df, 
    index=['success_expect', 'gender', 'frst_in_family', 'school_urbanicity'],
    columns=['intervention'],
    values='intervention',
    fill_value=0
)

# Display information about the pivot table
print("Bins with no control units:")
display(bin_pivot[bin_pivot[0] == 0])

print("Bins with no treatment units:")
display(bin_pivot[bin_pivot[1] == 0])



# References

- Yeager, D. S. et al. (2019). A national experiment reveals where a growth mindset improves achievement. Nature.
- Athey, S., & Wager, S. (2019). Estimating treatment effects with causal forests: An application. Observational studies.
- Facure, M. (2023). Causal Inference for the Brave and the True.